# Prepare Observational Data
Download and clean observational data so that we can compute timelags from it and compare to simulations. 

We need to do the following tasks for each map:
1. Download the level 1 AIA data for a given timestep and channel
2. Convert the level 1 data to level 1.5 data
3. Derotate the data to the time at which the AR was observed by Warren et al. (2012)
4. Cutout data around selected AR

At some stage, we should also delete the full sun image.

In [3]:
import os
import glob
import datetime

import numpy as np
from scipy.ndimage.interpolation import shift
import matplotlib.pyplot as plt
import drms
import dask
import distributed
import astropy.units as u
from astropy.coordinates import SkyCoord
from sunpy.map import Map
from sunpy.instr.aia import aiaprep
from sunpy.physics.differential_rotation import solar_rotate_coordinate

import synthesizAR

%matplotlib inline

In [4]:
client = distributed.Client()
client

Client Scheduler: tcp://127.0.0.1:36329 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 64 Cores: 64 Memory: 270.38 GB


First, set up the DRMS download client to get the AIA data from JSOC. This is best done from outside the notebook as it takes quite a while to download all of the data.

In [5]:
DOWNLOAD_DIR = '/storage-home/w/wtb2/data/timelag_synthesis_v2/observational_data/aia'
CUTOUTS_DIR = os.path.join(DOWNLOAD_DIR, 'cutouts')

Create a few methods for prepping the data. First, we pull in our observed magnetogram to get the right observing date, center, and observer coordinate as well as the appropriate FOV for the active region.

In [6]:
magnetogram = Map(
    '/storage-home/w/wtb2/data/timelag_synthesis_v2/observational_data/hmi/hmi_m_45s_2011_02_12_15_33_45_tai_magnetogram.fits'
)

In [7]:
reference_date = magnetogram.date
reference_center = magnetogram.center
reference_observer = magnetogram.observer_coordinate
cutout_left_corner = u.Quantity([-440,-375],u.arcsec)
cutout_right_corner = u.Quantity([-140,-75],u.arcsec)

Standard AIA prep procedure to get the data to all have a common scale.

In [8]:
def aiaprep_wrapper(filename):
    return aiaprep(Map(filename))

Derotate all the data to the time when the AR was observed by HMI as in Warren et al. (2012)

In [9]:
def derotate(aia_map):
    new_coord = solar_rotate_coordinate(aia_map.center, reference_date,
                                        new_observer_location=reference_observer)
    xs_pixel = (new_coord.Tx - reference_center.Tx)/aia_map.scale.axis1
    ys_pixel = (new_coord.Ty - reference_center.Ty)/aia_map.scale.axis2
    shifted_data = shift(aia_map.data, [ys_pixel.value, xs_pixel.value])
    derotated_map = Map(shifted_data, aia_map.meta)
    return derotated_map

Create a cutout around the AR in question

In [10]:
def cutout(aia_map):
    lc = SkyCoord(*cutout_left_corner, frame=aia_map.coordinate_frame)
    rc = SkyCoord(*cutout_right_corner, frame=aia_map.coordinate_frame)
    return aia_map.submap(lc, rc)

Save all of the data (and optionally delete the original image.

In [11]:
def save_and_cleanup(aia_cutout_map):
    fn = os.path.join(CUTOUTS_DIR, '{detector}_lev1.5_{date}_{channel}_cutout.fits'.format(
            detector=aia_cutout_map.meta['detector'].lower(), 
            date=aia_cutout_map.date.strftime('%Y%m%dT%H%M%S'),
            channel=aia_cutout_map.meta['wavelnth']))
    aia_cutout_map.save(fn)
    #os.remove(aia_full_filename)
    return fn

Now, we submit the jobs for each map to the Dask scheduler. The map is prepped, derotated, and cropped to the correct AR.

In [14]:
futures = []
map_filenames = sorted(glob.glob(os.path.join(DOWNLOAD_DIR, f'*.image*.fits')))
if not os.path.exists(CUTOUTS_DIR):
    os.mkdir(CUTOUTS_DIR)
for mf in map_filenames:
    lvl15_map = client.submit(aiaprep_wrapper, mf)
    derot_map = client.submit(derotate, lvl15_map)
    cutout_map = client.submit(cutout, derot_map)
    futures.append(client.submit(save_and_cleanup, cutout_map))

## Sandbox

In [29]:
channels = [94, 131, 171, 193, 211, 335]

In [31]:
time_min = None
time_max = None
for c in channels:
    tmp_list = sorted(glob.glob(os.path.join(DOWNLOAD_DIR, f'*.{c}.image*.fits')))
    tmp = Map(tmp_list[0])
    if time_min is None or tmp.date < time_min:
        time_min = tmp.date
    tmp = Map(tmp_list[-1])
    if time_max is None or tmp.date > time_max:
        time_max = tmp.date

In [37]:
(time_max - time_min).seconds

43199

In [ ]:
times = []
for i,f in enumerate(foo):
    times.append(datetime.datetime.strptime(os.path.basename(f).split('.')[2], '%Y%m%dT%H%M%SZ'))

In [ ]:
deltas = []
for i in range(len(foo) - 1):
    deltas.append((times[i+1] - times[i]).seconds)

In [ ]:
plt.plot(deltas,'.')
plt.xlim(3420,3450)

In [ ]:
print(foo[3439])
print(foo[3440])
print(foo[3441])

In [ ]:
q = c.export(f'aia.lev1_euv_12s[2011-02-12T21:00:00/1m@{cadence}][335]{{image}}',
             method='url_quick')

In [ ]:
q.data

In [ ]:
Map('/storage-home/w/wtb2/data/timelag_synthesis_v2/observational_data/aia/cutouts/aia_lev1.5_20110212T093214_94_cutout.fits').peek(vmin=0,vmax=150)
plt.show()

In [28]:
foo = Map('/storage-home/w/wtb2/data/timelag_synthesis_v2/observational_data/aia/cutouts/aia_lev1.5_20110212T09')
foo.peek()
plt.show()

ValueError: File not found or invalid input

In [22]:
foo.dimensions

PixelPair(x=<Quantity 500.0 pix>, y=<Quantity 500.0 pix>)

In [27]:
(cutout_right_corner[1] - cutout_left_corner[1])/0.6

<Quantity 500.0 arcsec>